In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import time

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1uS9uzODV0TTckQvLn8WK_EJCUhObsoMm' #use the link for the file 'taxi_model.csv' uploaded in your drive

In [0]:
fluff, id = link.split('=')
print (id)

1uS9uzODV0TTckQvLn8WK_EJCUhObsoMm


In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('taxi_model.csv')  
taxi = pd.read_csv('taxi_model.csv')

In [0]:
#taxi.drop(columns=['Unnamed: 0.1','Unnamed: 0.1.1'],inplace=True)
taxi.drop(columns=['Unnamed: 0'],inplace=True)


In [0]:
taxi.head()

,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,trip_total,trip_date,trip_timestamp,trip_hour,trip_day,weekend
0,180.0,0.4,24.0,24.0,4.50,2016-01-13,06:15:00,1,Wednesday,0
1,480.0,1.6,32.0,8.0,7.75,2016-01-18,15:45:00,3,Monday,0
2,120.0,0.2,8.0,8.0,5.00,2016-01-13,16:00:00,4,Wednesday,0
3,300.0,0.5,8.0,8.0,9.25,2016-01-18,15:15:00,3,Monday,0
4,1800.0,5.1,6.0,28.0,21.90,2016-01-20,08:30:00,2,Wednesday,0


In [0]:

taxi_sum = (taxi.groupby('pickup_community_area').sum().reset_index())
taxi_sum = taxi_sum[taxi_sum['trip_total']>=10**6]
taxi['trip_hour'] //=4

In [0]:
taxi = taxi.loc[taxi['pickup_community_area'].isin(taxi_sum['pickup_community_area'])]
taxi = taxi.loc[taxi['dropoff_community_area'].isin(taxi_sum['pickup_community_area'])]
taxi = taxi.reset_index()
taxi.drop(columns='index',inplace=True)

In [0]:
from sklearn.model_selection import train_test_split
X = taxi.loc[:,['trip_miles','pickup_community_area','dropoff_community_area','trip_hour','weekend']].values
y = taxi.loc[:,['trip_total']].values

In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
ohe = OneHotEncoder(categorical_features=[1,2,3],sparse=False)
X = ohe.fit_transform(X)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [0]:
def baseline_model():
    model = Sequential()
    model.add(Dense(13, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error',optimizer='adam',metrics =  )
    return model

In [0]:
np.random.seed(7)
estimators = []
estimators.append(('standardize',StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn=baseline_model,epochs=123,batch_size=8192,verbose=0)))
pipeline = Pipeline(estimators)


In [0]:
#keras neural network building
start = time.process_time()
kfold = KFold(n_splits=10,random_state=7)
results = cross_val_score(pipeline,X,y,cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
print(time.process_time()-start)







Results: -1012.19 (391.56) MSE
3040.118852926
